## Dissect Pipeline Data

Process to dissect the NG pipeline data into its elemental components for further analyses. This include breaking the dataset down into vertices tagged by the original OID which can be x-referenced with its attributes. 

In [47]:
#Imports
import arcpy, os, zipfile, zlib

#Set environments
arcpy.env.workspace = '..\\scratch'
arcpy.env.scratchWorkspace = '..\\scratch'
arcpy.env.overwriteOutput = True

In [3]:
#Read in the pipeline data
gdb_NG = '\\\\ns-leviathan.win.duke.edu\\biogas_GIS\\Rextag_Zone2\\Rextag_Zone2_Natural_Gas.gdb\\Natural_Gas'
fc_NaturalGasPipelines =  os.path.join(gdb_NG,'NaturalGasPipelines')
if not arcpy.Exists(fc_NaturalGasPipelines):
    print("Error finding pipeline dataset!")
else:
    sr = arcpy.Describe(fc_NaturalGasPipelines).spatialReference
    srName = sr.name
    srCode = sr.GCSCode
    arcpy.env.outputCoordinateSystem=sr
    print("Natural gas pipeline feature class found.")
    print("Spatial reference set to {} (code:{})".format(srName,srCode))

Natural gas pipeline feature class found.
Spatial reference set to GCS_North_American_1983 (code:4269)


In [22]:
#Isolate NC Pipelines
theState = "North Carolina"
fc_StatePipelines = arcpy.Select_analysis(in_features=fc_NaturalGasPipelines,
                                          out_feature_class='statePipes.shp',
                                          where_clause="STATE_NAME = '{}'".format(theState)).getOutput(0)

In [24]:
#Convert to feature class of verticies
fc_StatePoints = arcpy.FeatureVerticesToPoints_management(in_features=fc_StatePipelines,
                                                          out_feature_class='statePoints',
                                                          point_location='ALL').getOutput(0)

In [27]:
#Convert features to JSON
json_Vertices = arcpy.FeaturesToJSON_conversion(in_features=fc_StatePoints,
                                                out_json_file='../scratch/NC_Vertices.geosjon',
                                                format_json='FORMATTED',
                                                geoJSON='GEOJSON').getOutput(0)

In [50]:
json_Vertices

'..\\scratch\\NC_Vertices.geojson'

In [52]:
#Zip the file geojson file
zf = zipfile.ZipFile('../scratch/NC_Verticies.zip', mode='w')
zf.write(json_Vertices,compress_type=zipfile.ZIP_DEFLATED)
zf.close()
